# 1B_mg_preprocess_hichip

06/03/2020

In [1]:
import pandas as pd
import pybedtools
import sys, re
import os,glob
import numpy as np
import subprocess
from collections import defaultdict,Counter

In [2]:
prom_bed = pybedtools.BedTool('../data/external/promoter_hg19_2000_500_sort.bed')
prom_bed.count()


20260

In [72]:
valid_chr=[str(x) for x in list(range(1,23))]+['X','Y']
valid_chr_2 = ['chr'+x for x in valid_chr]
valid_chr = valid_chr+valid_chr_2
def read_bedpe_file(filepath,anchor_width=5000,valid_chr=valid_chr):
    results = defaultdict(int)
    with open(filepath) as f:
    #     result_idx=0
        for idx, line in enumerate(f.readlines()):
            #DEBUG
    #         if idx>100:
    #             break
            if (idx % 500000)==0:
                print(idx, len(results))
            chr_x, start_x , stop_x,  chr_y , start_y , stop_y , etc , count  = line.strip().split(' ')
        
            # filter validity
            if chr_x not in valid_chr:
                #print('chr_x',chr_x)
                continue
            if chr_y not in valid_chr:
                #print('chr_y',chr_y)
                continue
            
            
            #rounding to anchor width
            if not chr_x.startswith('chr'):
                chr_x = 'chr'+chr_x
            start_x = round(int(start_x)/anchor_width)*anchor_width
            stop_x = max(start_x+anchor_width,round(int(stop_x)/anchor_width)*anchor_width)
            if not chr_y.startswith('chr'):
                chr_y = 'chr'+chr_y
            start_y = round(int(start_y)/anchor_width)*anchor_width
            stop_y = max(start_y+anchor_width,round(int(stop_y)/anchor_width)*anchor_width)

            #partitioning to anchor width (deal with large fragments)
            anchors_x=[]
            for x in range(start_x, stop_x,anchor_width):
                anchors_x.append(chr_x+'_' + str(x) +"_"+str(x+anchor_width))
            anchors_y=[]
            for x in range(start_y, stop_y,anchor_width):
                anchors_y.append(chr_y+'_' + str(x) +"_"+str(x+anchor_width))

            for source in anchors_x:
                for target in anchors_y:
                    results[(source,target)] += int(count)
#                     results[(target,source)] += int(count)

    loop_df = pd.concat([pd.DataFrame(results.keys()),pd.DataFrame(results.values())],axis=1)
    loop_df.columns=['source','target','count']
    return loop_df


def hichip_to_anchor(hichip_df, sample):
    """
    hichip_df<DataFrame> from `hichip_to_df`
    """
    anchors_list = sorted(set(hichip_df.source).union(set(hichip_df.target)))
    anchors_df = pd.DataFrame({"anchors":anchors_list})
    anchors_df_split = anchors_df.anchors.str.split('_', expand=True)
    anchors_df_split.columns = ['chr', 'start', 'end']
    anchors_df = pd.concat([anchors_df, anchors_df_split], axis=1)
    anchors_df['sample']= sample
    anchors_df = anchors_df[['chr','start','end','anchors','sample']]
    return anchors_df


def filter_through_tss(anchor_df, prom_bed):
    anchor_bed = pybedtools.BedTool.from_dataframe(anchor_df).sort()
    return anchor_bed.intersect(prom_bed, wa=True, wb=False, sorted=True).to_dataframe()

In [91]:
# read file
input_filepath = '../data/raw/hichip/'
output_filepath = '../data/interim/merged/'
extension='.bedpe'
type_prefix=''


In [92]:
# set up file paths
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

loops_dir = os.path.join(output_filepath, 'loops')
anchors_dir = os.path.join(output_filepath, 'anchors')
anchors_bed_dir = os.path.join(output_filepath, 'anchors_bed')
if not os.path.exists(loops_dir):
    os.makedirs(loops_dir)
    os.makedirs(anchors_dir)
    os.makedirs(anchors_bed_dir)

In [93]:
%%time
tissue_loop_dict = {}
for subdir, dirs, files in os.walk(input_filepath):
    tissue = os.path.basename(subdir)
    file_idx = 0
    print('merging and csv...', tissue)
    
    
#     #DEBUG
#     if tissue!='H9_D28':
#         continue
#     merged_loop_df = pd.DataFrame()
#     merged_anchor_df = pd.DataFrame()

    # loop through tissue folder
    for filename in sorted(files):

        input_filepath = os.path.join(subdir, filename )# bedpe file
        output_filepath_tissue = subdir

        if filename.endswith(extension) :
            print(input_filepath)
            loop_df = read_bedpe_file(input_filepath)
            print('read loop_df',loop_df.shape)
            anchors_df = hichip_to_anchor(loop_df,tissue)
            print('read anchor_df',anchors_df.shape)
            
            
            
            if file_idx==0:
                merged_loop_df = loop_df
                merged_anchor_df = anchors_df
                file_idx+=1
                print('start',merged_loop_df.shape, merged_anchor_df.shape)
#                 merged_anchor_df = anchors_df
            else:
                print('merging')
                merged_loop_df = pd.concat([merged_loop_df, loop_df],ignore_index=True)
                merged_loop_df = merged_loop_df.groupby(['source', 'target']).sum()
                merged_loop_df.reset_index(inplace=True)
                
                merged_anchor_df = pd.concat([merged_anchor_df, anchors_df],ignore_index=True)
                merged_anchor_df.drop_duplicates(subset=['chr','start', 'end'],inplace=True)
                merged_anchor_df.reset_index(drop=True,inplace=True)

                file_idx+=1
                print('merging done',merged_loop_df.shape,merged_anchor_df.shape)


    
    # end loop through tissue folder
    if file_idx>0:
        print('end loop through tissue folder, now filtering')
        merged_anchor_df_P = filter_through_tss(merged_anchor_df,prom_bed)
        merged_anchor_df_P.columns = ['chr','start','end','anchors','sample']
        anchors_P = merged_anchor_df_P.anchors.unique()
        merged_loop_df['source_P'] = merged_loop_df.source.isin(anchors_P)
        merged_loop_df['target_P'] = merged_loop_df.target.isin(anchors_P)
        merged_loop_df['loop_P'] = merged_loop_df['source_P']  | merged_loop_df['target_P']
        merged_loop_df = merged_loop_df[merged_loop_df.loop_P][['source','target','count']]
        
        merged_anchor_df= hichip_to_anchor(merged_loop_df,tissue)
        print('filtering done',tissue, merged_loop_df.shape,merged_anchor_df.shape)


        if merged_loop_df.shape[0]>0:
            merged_loop_df.to_csv(os.path.join(loops_dir, tissue+type_prefix+'.loops.csv'))
            merged_anchor_df.to_csv(os.path.join(anchors_dir, tissue+type_prefix+'.anchors.csv'))
            merged_anchor_df = merged_anchor_df[['chr','start','end', 'anchors']]
            merged_anchor_df.to_csv(os.path.join(anchors_bed_dir, tissue+type_prefix+'.anchors.bed'),sep='\t', index=False, header=False)



merging and csv... 
merging and csv... SL_D2
../data/raw/hichip/SL_D2/SLNgn2-20.filt.intra.loop_counts.bedpe
0 0
500000 1770771
1000000 3556018
1500000 5298088
2000000 7150031
2500000 8975736
3000000 11166873
3500000 13180643
4000000 14765495
4500000 16204844
5000000 17758280
5500000 19683472
6000000 21606650
6500000 23898889
7000000 25540922
7500000 27546033
8000000 29683305
8500000 32137864
9000000 34647062
9500000 36816361
10000000 39129203
10500000 41232868
11000000 43638499
11500000 45707002
12000000 47731925
12500000 49771495
13000000 51804079
read loop_df (53032965, 3)
read anchor_df (406093, 5)
start (53032965, 3) (406093, 5)
../data/raw/hichip/SL_D2/SLNgn2-21.filt.intra.loop_counts.bedpe
0 0
500000 1720545
1000000 3489628
1500000 5132450
2000000 6840636
2500000 8562141
3000000 10633053
3500000 12593272
4000000 14129082
4500000 15557478
5000000 17036470
5500000 18904301
6000000 20533056
6500000 22711662
7000000 24428644
7500000 26331949
8000000 28341054
8500000 30470319
9000000

# washu tract

### Step 0:
read over bed_longrange_formatting.py 

longrange
The longrange track is a bed format-like file type. Each row contains columns from left to right: chromosome, start position (0-based), and end position (not included), interaction target in this format chr2:333-444,55. As an example, interval “chr1:111-222” interacts with interval “chr2:333-444” on a score of 55, we will use following two lines to represent this interaction:
```
chr1    111 222  chr2:333-444,55
chr2    333 444  chr1:111-222,55
```
Important:
- Be sure to make TWO records for a pair of interacting loci, one record for each locus.
- check the bed files to see if 'chr' is a prefix,

### Step 1: make longrange files
- longrange format (see above)
- sort bed file
- save sorted bed file
- bgzip file
- tabix to create index


In [ ]:
save_dir = '../data/processed/washu/hichip'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [126]:
%%time
longrange_dir = '../data/'
for loop_file in glob.glob('../data/interim/merged/loops/*'):
    # read in looping file and convert to longrange format
    tissue = os.path.basename(loop_file).split('.')[0]
    print(tissue, loop_file)
    loop_df = pd.read_csv(loop_file, index_col=0)
    loop_df_rev = loop_df.copy()
    loop_df_rev = loop_df_rev[['target','source','count']]
    loop_df_rev.columns = ['source','target','count']
    loop_df_bi = pd.concat([loop_df, loop_df_rev],axis=0)
    loop_df_bi[['chr','start','end']] = loop_df_bi.source.str.split('_',expand=True)
    loop_df_bi[['chr2','start2','end2']] = loop_df_bi.target.str.split('_',expand=True)
    loop_df_bi['target'] = loop_df_bi.chr2 + ':' + loop_df_bi.start2 + '-' + loop_df_bi.end2 + ',' + loop_df_bi['count'].map(str)
    loop_df_bi = loop_df_bi[['chr','start','end','target']]
    
    # make bed file, sort, and save
    loop_bed = pybedtools.BedTool.from_dataframe(loop_df_bi).sort()
    output_filepath = os.path.join(save_dir, tissue+'_longrange.bed')
    loop_bed.saveas(output_filepath)
    
    # bgzip bedfile
    subprocess.Popen(['bgzip', output_filepath])
    output_filepath_bzip = output_filepath+'.gz'
    
    # tabix create index
    subprocess.Popen(['tabix', '-p', 'bed', output_filepath_bzip])

SL_D2 ../data/interim/merged/loops/SL_D2.loops.csv


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Astrocytes ../data/interim/merged/loops/Astrocytes.loops.csv


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SLC_D2 ../data/interim/merged/loops/SLC_D2.loops.csv


...

/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


H9_D10 ../data/interim/merged/loops/H9_D10.loops.csv
H9_D2 ../data/interim/merged/loops/H9_D2.loops.csv


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


H9_D28 ../data/interim/merged/loops/H9_D28.loops.csv


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Step 2: make hub.config.json

keep in same folder

```
[
  {
    "filename": "H9_D0_longrange.bed.gz",
    "type":"longrange",
    "name": "H9_D0",
    "label":"H9_D0",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"H9_D0",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
  {
    "filename": "H9_D2_longrange.bed.gz",
    "type":"longrange",
    "name": "H9_D2",
    "label":"H9_D2",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"H9_D2",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
    {
    "filename": "H9_D4_longrange.bed.gz",
    "type":"longrange",
    "name": "H9_D4",
    "label":"H9_D4",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"H9_D4",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
  {
    "filename": "H9_D10_longrange.bed.gz",
    "type":"longrange",
    "name": "H9_D10",
    "label":"H9_D10",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"H9_D10",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
  {
    "filename": "H9_D28_longrange.bed.gz",
    "type":"longrange",
    "name": "H9_D28",
    "label":"H9_D28",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"H9_D28",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
    {
    "filename": "SL_D0_longrange.bed.gz",
    "type":"longrange",
    "name": "SL_D0",
    "label":"SL_D0",
    "options":{
     "color":"#8D4ECC",
     "color2":"#004080",
     "displayMode":"arc",
     "label":"SL_D0",
     "scoreMax":40,
     "scoreMin":10,
     "scoreScale":"fixed"
    }
  },
      {
        "filename": "SL_D2_longrange.bed.gz",
      	"type":"longrange",
        "name": "SL_D2",
     	"label":"SL_D2",
      "options":{
         "color":"#8D4ECC",
         "color2":"#004080",
         "displayMode":"arc",
         "label":"SL_D2",
         "scoreMax":40,
         "scoreMin":10,
         "scoreScale":"fixed"
      }
    },
    {
      "filename": "SLC_D0_longrange.bed.gz",
      "type":"longrange",
      "name": "SLC_D0",
      "label":"SLC_D0",
      "options":{
       "color":"#8D4ECC",
       "color2":"#004080",
       "displayMode":"arc",
       "label":"SLC_D0",
       "scoreMax":40,
       "scoreMin":10,
       "scoreScale":"fixed"
      }
    },
      {
        "filename": "SLC_D2_longrange.bed.gz",
      	"type":"longrange",
        "name": "SLC_D2",
     	"label":"SLC_D2",
      "options":{
         "color":"#8D4ECC",
         "color2":"#004080",
         "displayMode":"arc",
         "label":"SLC_D2",
         "scoreMax":40,
         "scoreMin":10,
         "scoreScale":"fixed"
      }
    }

]
      }
    },
]
```

# Longrange (thres)
- and normalize
take

In [32]:
save_dir = '../data/processed/washu/hichip_thres_adjust'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [5]:
THRES=10

In [30]:
loop_file = '../data/interim/merged/loops/H9_D0.loops.csv'
loop_df = pd.read_csv(loop_file, index_col=0)

In [49]:
subprocess.Popen(['tabix', '-p', 'bed', '../data/processed/washu/hichip/SLC_D2_longrange.bed.gz'])

In [33]:
%%time
longrange_dir = '../data/'
for loop_file in glob.glob('../data/interim/merged/loops/*'):
    # read in looping file and convert to longrange format
    tissue = os.path.basename(loop_file).split('.')[0]
    loop_df = pd.read_csv(loop_file, index_col=0)
    thres = np.percentile(loop_df['count'],95)#.describe()#.iloc
    loop_df = loop_df[loop_df['count']>thres]
    print(tissue, loop_file,loop_df.shape, thres)
    loop_df_rev = loop_df.copy()
    loop_df_rev = loop_df_rev[['target','source','count']]
    loop_df_rev.columns = ['source','target','count']
    loop_df_bi = pd.concat([loop_df, loop_df_rev],axis=0)
    loop_df_bi[['chr','start','end']] = loop_df_bi.source.str.split('_',expand=True)
    loop_df_bi[['chr2','start2','end2']] = loop_df_bi.target.str.split('_',expand=True)
    loop_df_bi['target'] = loop_df_bi.chr2 + ':' + loop_df_bi.start2 + '-' + loop_df_bi.end2 + ',' + loop_df_bi['count'].map(str)
    loop_df_bi = loop_df_bi[['chr','start','end','target']]
    
    # make bed file, sort, and save
    loop_bed = pybedtools.BedTool.from_dataframe(loop_df_bi).sort()
    output_filepath = os.path.join(save_dir, tissue+'_longrange.bed')
    loop_bed.saveas(output_filepath)
    
    # bgzip bedfile
    subprocess.Popen(['bgzip', output_filepath])
    output_filepath_bzip = output_filepath+'.gz'
    
    # tabix create index
    subprocess.Popen(['tabix', '-p', 'bed', output_filepath_bzip])

SL_D2 ../data/interim/merged/loops/SL_D2.loops.csv (260004, 3) 20.0


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Astrocytes ../data/interim/merged/loops/Astrocytes.loops.csv (86821, 3) 13.0


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SLC_D2 ../data/interim/merged/loops/SLC_D2.loops.csv (205409, 3) 15.0


...

SLC_D0 ../data/interim/merged/loops/SLC_D0.loops.csv (120695, 3) 11.0
H9_D10 ../data/interim/merged/loops/H9_D10.loops.csv (34846, 3) 5.0


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


H9_D2 ../data/interim/merged/loops/H9_D2.loops.csv (333897, 3) 30.0


/Users/mguo123/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


H9_D28 ../data/interim/merged/loops/H9_D28.loops.csv (45081, 3) 4.0
CPU times: user 57.2 s, sys: 2.87 s, total: 1min
Wall time: 44.7 s


In [86]:
%%time
tissue_loop_dict = {}
for subdir, dirs, files in os.walk(input_filepath):
    tissue = os.path.basename(subdir)
    file_idx = 0
    print('merging and csv...', tissue)
    
    
    #DEBUG
    if tissue!='H9_D28':
        continue
#     merged_loop_df = pd.DataFrame()
#     merged_anchor_df = pd.DataFrame()

    # loop through tissue folder
    for filename in sorted(files):

        input_filepath = os.path.join(subdir, filename )# bedpe file
        output_filepath_tissue = subdir

        if filename.endswith(extension) :
            print(input_filepath)
            loop_df = read_bedpe_file(input_filepath)
            loop_df = filter_through_tss(loop_df,prom_bed)
            print('read loop_df',loop_df.shape)
            if file_idx==0:
                merged_loop_df = loop_df
                file_idx+=1
                print('start',merged_loop_df.shape)
#                 merged_anchor_df = anchors_df
            else:
                print('merging')
                merged_loop_df = pd.concat([merged_loop_df, loop_df],ignore_index=True)
                merged_loop_df = merged_loop_df.groupby(['source', 'target']).sum()
                merged_loop_df.reset_index(inplace=True)
                file_idx+=1
                print('merging done',merged_loop_df.shape)

#                 merged_anchor_df = pd.concat([merged_anchor_df, anchors_df],ignore_index=True)
#                 merged_anchor_df.drop_duplicates(subset=['chr','start', 'end'],inplace=True)
#                 merged_anchor_df.reset_index(drop=True,inplace=True)
    
    
    # end loop through tissue folder
            

In [22]:
%%time
anchor_width = 5000
valid_chr = [str(x) for x in list(range(1,22))]+['X','Y']
with open('../data/raw/hichip/H9_D0/H9-B1.filt.intra.loop_counts.bedpe', 'r') as f:
    results = defaultdict(int)
#     result_idx=0
    for idx, line in enumerate(f.readlines()):
        #DEBUG
#         if idx>100:
#             break
        if (idx % 500000)==0:
            print(idx)
        chr_x, start_x , stop_x,  chr_y , start_y , stop_y , etc , count  = line.strip().split(' ')
        
        if chr_x not in valid_chr:
            continue
        if chr_y not in valid_chr:
            continue
        
        #rounding to anchor width
        chr_x = 'chr'+chr_x
        start_x = round(int(start_x)/anchor_width)*anchor_width
        stop_x = max(start_x+anchor_width,round(int(stop_x)/anchor_width)*anchor_width)
        chr_y = 'chr'+chr_y
        start_y = round(int(start_y)/anchor_width)*anchor_width
        stop_y = max(start_y+anchor_width,round(int(stop_y)/anchor_width)*anchor_width)

        #partitioning to anchor width (deal with large fragments)
        anchors_x=[]
        for x in range(start_x, stop_x,anchor_width):
            anchors_x.append(chr_x+'_' + str(x) +"_"+str(x+anchor_width))
        anchors_y=[]
        for x in range(start_y, stop_y,anchor_width):
            anchors_y.append(chr_y+'_' + str(x) +"_"+str(x+anchor_width))
        
        for source in anchors_x:
            for target in anchors_y:
                results[(source,target)] += int(count)
                results[(target,source)] += int(count)
            
loop_df = pd.concat([pd.DataFrame(results.keys()),pd.DataFrame(results.values())],axis=1)
loop_df.columns=['source','target','count']
